In [2]:
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from rw_formula import RW_Calc

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [    
    Tool(
        name="rwa_db",
        func=db_chain.run,
        description="useful for quering a database"
    )
]

In [8]:
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True, 
    handle_parsing_errors=True
)

In [6]:
agent.run("Report the total RWA and EAD per SEGMENT from RWA_122022 table")



> Entering new SQLDatabaseChain chain...
SELECT SEGMENT, SUM(RWA), SUM(EAD) FROM RWA_122022 GROUP BY SEGMENT
SQLQuery:SELECT "SEGMENT", SUM("RWA"), SUM("EAD") FROM "RWA_122022" GROUP BY "SEGMENT"
SQLResult: [('Corporate', 85075812.12498187, 156657332.34921697), ('Retail', 37806680.90001285, 165698955.81288758)]
Answer:Segment: Corporate
Total RWA: 85,075,812.12
Total EAD: 156,657,332.35

Segment: Retail
Total RWA: 37,806,680.90
Total EAD: 165,698,955.81
> Finished chain.


'The total RWA and EAD per segment from the RWA_122022 table are as follows:\n- Segment: Corporate\n  - Total RWA: 85,075,812.12\n  - Total EAD: 156,657,332.35\n- Segment: Retail\n  - Total RWA: 37,806,680.90\n  - Total EAD: 165,698,955.81'

In [9]:
agent.run("What is the sum of all EXPOSURE in RWA_122022 for all ACCOUNT_NUMBER whose STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table")



> Entering new AgentExecutor chain...
I need to compare the STATUS column between the RWA_122022 and RWA_032023 tables to find the ACCOUNT_NUMBER that changed from Performing to Non Performing. Then, I need to calculate the sum of the EXPOSURE for those ACCOUNT_NUMBER in the RWA_122022 table.
Action: rwa_db
Action Input: SELECT ACCOUNT_NUMBER FROM RWA_122022 WHERE STATUS = 'Performing' AND ACCOUNT_NUMBER IN (SELECT ACCOUNT_NUMBER FROM RWA_032023 WHERE STATUS = 'Non Performing')

> Entering new SQLDatabaseChain chain...
SELECT ACCOUNT_NUMBER FROM RWA_122022 WHERE STATUS = 'Performing' AND ACCOUNT_NUMBER IN (SELECT ACCOUNT_NUMBER FROM RWA_032023 WHERE STATUS = 'Non Performing')
SQLQuery:SELECT "ACCOUNT_NUMBER" FROM "RWA_122022" WHERE "STATUS" = 'Performing' AND "ACCOUNT_NUMBER" IN (SELECT "ACCOUNT_NUMBER" FROM "RWA_032023" WHERE "STATUS" = 'Non Performing')
SQLResult: [('9949846894mqqb38',), ('8589346566omjm76',), ('3939009254tfzg85',), ('6572063347hjeb20',), ('8888826809jvcg55',), ('2

'The sum of all EXPOSURE in RWA_122022 for all ACCOUNT_NUMBER whose STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table is 575151.3173016.'

In [10]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="risk weight calculator",
        func=RW_Calc,
        description=
        """calculates a loan risk weight give the argulamts: 
        Segment - Possible values are 'Bank', 'Corporate', and 'Retail'), 
        PD - Probability of Default
        LGD - Loss Given Default
        m - Remaining maturity of the loan in years
        Large_Fin - If 'Y' the client is a Flag for Large Financial Institution, otherwise 'N'
        size - size of the client in MEUR, usually this is the client's turnover
        mortgage - If 'Y' the exposure is a mortgage loan, otherwise 'N'
        revolving - If 'Y' the exposure is a revolving loan, otherwise 'N'
        """
    ),    
    Tool(
        name="rwa_db",
        func=db_chain.run,
        description="useful for quering a database"
    )
]

In [11]:
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True, 
    handle_parsing_errors=True
)

In [12]:
agent.run("Read the information for Client 'sfman997ux8036' in RWA_122022 table and then calculate the RW if PD doubles")



> Entering new AgentExecutor chain...
I need to query the RWA_122022 table to get the information for Client 'sfman997ux8036'. Then I can use the risk weight calculator to calculate the RW if PD doubles.
Action: rwa_db
Action Input: SELECT * FROM RWA_122022 WHERE Client = 'sfman997ux8036'

> Entering new SQLDatabaseChain chain...
SELECT * FROM RWA_122022 WHERE Client = 'sfman997ux8036'
SQLQuery:SELECT * FROM RWA_122022 WHERE CLIENT_NUMBER = 'sfman997ux8036'
SQLResult: [('sfman997ux8036', '1324710203qiph20', 'CREDIT_CARD', 'Performing', 'Retail', 0.0, 18728.13876, 18728.13876, 11004.65713, 0.0, 0.0246518, 0.3092, 42.58029755, 1.15616438, 'N', 'Y', 'N', 0.245013691, 2696.291657)]
Answer:CLIENT_NUMBER: sfman997ux8036
ACCOUNT_NUMBER: 1324710203qiph20
PRODUCT: CREDIT_CARD
STATUS: Performing
SEGMENT: Retail
EXPOSURE_DRAWN: 0.0
EXPOSURE_UNDRAWN: 18728.13876
EXPOSURE: 18728.13876
EAD: 11004.65713
EAD_COLL: 0.0
PD: 0.0246518
LGD: 0.3092
SIZE: 42.58029755
MATURITY: 1.15616438
F_MORTGAGE: N
F_R

'0.3950618648985617'